In [1]:
!gdown 1gj5-HatoF4n6V_RYB2r9vYpeawJvgf2-
!gdown 17twDjhPHlpV9ONl_Rekg_BuFLdnZLUnq

Downloading...
From: https://drive.google.com/uc?id=1gj5-HatoF4n6V_RYB2r9vYpeawJvgf2-
To: /Users/tada/Documents/GitHub/data-warehouse-mining/src/assgn01_etlAndDataProcessing/ETLDataSource1-286218-17243904498090.xlsx
100%|██████████████████████████████████████| 15.3k/15.3k [00:00<00:00, 25.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=17twDjhPHlpV9ONl_Rekg_BuFLdnZLUnq
To: /Users/tada/Documents/GitHub/data-warehouse-mining/src/assgn01_etlAndDataProcessing/ETLDataSource2-286218-17243904776879.xlsx
100%|██████████████████████████████████████| 12.7k/12.7k [00:00<00:00, 18.9MB/s]


In [2]:
import pandas as pd
from pandas import DataFrame, Series

rawOrderSource1: DataFrame = pd.read_excel(io='ETLDataSource1-286218-17243904498090.xlsx', sheet_name='orderSource1')
rawProductSource1: DataFrame = pd.read_excel(io='ETLDataSource1-286218-17243904498090.xlsx', sheet_name='productSource1')
rawStateLookupSource1: DataFrame = pd.read_excel(io='ETLDataSource1-286218-17243904498090.xlsx', sheet_name='StateLookup')

rawOrderSource2: DataFrame = pd.read_excel(io='ETLDataSource2-286218-17243904776879.xlsx', sheet_name='orderSource2')
rawProductSource2: DataFrame = pd.read_excel(io='ETLDataSource2-286218-17243904776879.xlsx', sheet_name='productSource2')

In [3]:
from pandas import StringDtype


stateLookup: dict[str, str] = {
    abbreviation: stateName
    for _, stateName, abbreviation
    in rawStateLookupSource1.itertuples()
    }

def getFirstName(fullName: str) -> str:
    return fullName.split()[0]

def getLastName(fullName: str) -> str:
    return fullName.split()[-1]

firstNameSource1: Series = (
    rawOrderSource1['CustomerName']
    .transform(getFirstName)
    .rename('CustomerFirstName')
    )

lastNameSource1: Series = (
    rawOrderSource1['CustomerName']
    .transform(getLastName)
    .rename('CustomerLastName')
    )

namesSource1: DataFrame = (
    rawOrderSource1['CustomerName']
    .transform(func=[getFirstName, getLastName])
)

In [4]:
processedSource1: DataFrame = (
    rawOrderSource1
    .merge(right=rawProductSource1, how='inner', on='OrderID')
    .replace(to_replace={'CustomerState': stateLookup})
    )

processedSource1.head()

,OrderID,CustomerName,CustomerCity,CustomerState,CustomerStatus,OrderDate,ProductID,Product,UnitPrice,Quantity,Discount,FullPrice,ExtendedPrice,TotalDiscount
0,10248,Suzan Plock,Pittsburgh,Pennsylvania,Silver,2023-01-11,11,Queso Cabrales,14.0,12,0.0,168.0,168.0,0.0
1,10248,Suzan Plock,Pittsburgh,Pennsylvania,Silver,2023-01-11,42,Singaporean Hokkien Fried Mee,9.8,10,0.0,98.0,98.0,0.0
2,10248,Suzan Plock,Pittsburgh,Pennsylvania,Silver,2023-01-11,72,Mozzarella di Giovanni,34.8,5,0.0,174.0,174.0,0.0
3,10249,Allan Strate,Miami,Florida,Platinum,2023-01-11,51,Manjimup Dried Apples,42.4,40,0.0,1696.0,1696.0,0.0
4,10249,Allan Strate,Miami,Florida,Platinum,2023-01-11,14,Tofu,18.6,9,0.0,167.4,167.4,0.0
